In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


data = pd.read_csv('bank-full-4.csv', sep=';')


selected_columns = [
    'age', 'job', 'marital', 'education', 'balance', 'housing',
    'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
    'previous', 'poutcome', 'y'
]
selected_data = data[selected_columns]


# Проверяем наличие пропущенных значений в каждом столбце
missing_values = selected_data.isnull().sum()
print("Пропущенные значения в данных:")
print(missing_values)

selected_data.head()

Пропущенные значения в данных:
age          0
job          0
marital      0
education    0
balance      0
housing      0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64


,age,job,marital,education,balance,housing,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,2143,yes,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,29,yes,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,2,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,1506,yes,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,1,no,unknown,5,may,198,1,-1,0,unknown,no


In [6]:
from sklearn.model_selection import train_test_split


# Splitting the data into train/validation/test sets (60%/20%/20%)
train_data, temp_data = train_test_split(selected_data, test_size=0.4, random_state=1)
validation_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=1)


len(train_data), len(validation_data), len(test_data)

(27126, 9042, 9043)

In [7]:
from sklearn.metrics import roc_auc_score

# Определение числовых переменных для анализа
numerical_features = ['balance', 'day', 'duration', 'previous']

# Подготовка переменных и целевой переменной (используем обучающую выборку)
X_train = train_data[numerical_features]
y_train = train_data['y'].apply(lambda x: 1 if x == 'yes' else 0)  # Преобразуем 'y' в числовой формат

# Словарь для хранения AUC значений
feature_auc = {}

# Вычисление ROC AUC для каждой переменной
for feature in numerical_features:
    auc = roc_auc_score(y_train, X_train[feature])
    # Инвертируем переменную, если AUC < 0.5
    if auc < 0.5:
        auc = roc_auc_score(y_train, -X_train[feature])
    feature_auc[feature] = auc

# Наивысший AUC
max_auc_feature = max(feature_auc, key=feature_auc.get)

# Вывод результатов
feature_auc, max_auc_feature


({'balance': np.float64(0.5916931412945473),
  'day': np.float64(0.5291275984648961),
  'duration': np.float64(0.8128648066681694),
  'previous': np.float64(0.5986704105715122)},
 'duration')

In [8]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# One-hot-encoding с использованием DictVectorizer
train_dicts = train_data.to_dict(orient='records')  # Преобразование данных в список словарей
val_dicts = validation_data.to_dict(orient='records')

vectorizer = DictVectorizer(sparse=False)
X_train = vectorizer.fit_transform(train_dicts)
X_val = vectorizer.transform(val_dicts)

# Целевая переменная
y_train = train_data['y'].apply(lambda x: 1 if x == 'yes' else 0)
y_val = validation_data['y'].apply(lambda x: 1 if x == 'yes' else 0)

# Логистическая регрессия
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000)
model.fit(X_train, y_train)

# Предсказания и вычисление AUC на валидационном наборе
y_pred_val = model.predict_proba(X_val)[:, 1]
auc_val = roc_auc_score(y_val, y_pred_val)

auc_val


np.float64(1.0)

In [10]:
# Вычисляем F1-score для порогов от 0.0 до 1.0 с шагом 0.01
thresholds = np.arange(0.0, 1.01, 0.01)
f1_scores = [f1_score(y_val, y_pred_val >= thresh) for thresh in thresholds]

# Находим порог с максимальным значением F1
max_f1_index = np.argmax(f1_scores)
max_f1_threshold = thresholds[max_f1_index]
max_f1_score = f1_scores[max_f1_index]

# Выводим результаты
print(f"Максимальный F1-score: {max_f1_score:.2f} при пороге: {max_f1_threshold:.2f}")


Максимальный F1-score: 1.00 при пороге: 0.08


In [11]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

# Инициализация KFold с 5 фолдами
kf = KFold(n_splits=5, shuffle=True, random_state=1)

# Список для хранения оценок AUC на каждом фолде
auc_scores = []

# Выполняем кросс-валидацию
for train_index, val_index in kf.split(X_train):
    # Делим данные на обучающую и валидационную выборки
    X_fold_train, X_fold_val = X_train[train_index], X_train[val_index]
    y_fold_train, y_fold_val = y_train.iloc[train_index], y_train.iloc[val_index]
    
    # Обучаем логистическую регрессию
    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000)
    model.fit(X_fold_train, y_fold_train)
    
    # Предсказания вероятностей и вычисление AUC
    y_pred_fold_val = model.predict_proba(X_fold_val)[:, 1]
    auc = roc_auc_score(y_fold_val, y_pred_fold_val)
    auc_scores.append(auc)

# Вычисляем стандартную ошибку (стандартное отклонение / sqrt(n))
std_error = np.std(auc_scores) / np.sqrt(len(auc_scores))

print(f"Стандартная ошибка оценок AUC: {std_error:.5f}")


Стандартная ошибка оценок AUC: 0.00000


In [12]:
# Перебираемые значения параметра C
C_values = [0.000001, 0.001, 1]

# Инициализация KFold с 5 фолдами
kf = KFold(n_splits=5, shuffle=True, random_state=1)

# Словарь для хранения результатов AUC для каждого значения C
results = {}

for C in C_values:
    auc_scores = []
    
    # Кросс-валидация для текущего значения C
    for train_index, val_index in kf.split(X_train):
        X_fold_train, X_fold_val = X_train[train_index], X_train[val_index]
        y_fold_train, y_fold_val = y_train.iloc[train_index], y_train.iloc[val_index]
        
        # Обучение модели с текущим C
        model = LogisticRegression(solver='liblinear', C=C, max_iter=1000)
        model.fit(X_fold_train, y_fold_train)
        
        # Предсказания и вычисление AUC
        y_pred_fold_val = model.predict_proba(X_fold_val)[:, 1]
        auc = roc_auc_score(y_fold_val, y_pred_fold_val)
        auc_scores.append(auc)
    
    # Вычисляем среднее AUC и стандартное отклонение
    mean_auc = np.round(np.mean(auc_scores), 3)
    std_auc = np.round(np.std(auc_scores), 3)
    
    results[C] = (mean_auc, std_auc)

# Находим лучший параметр C по максимальному среднему AUC
best_C = min(results, key=lambda x: (-results[x][0], results[x][1]))

# Выводим результаты
for C, (mean_auc, std_auc) in results.items():
    print(f"C: {C}, Средний AUC: {mean_auc}, Стандартное отклонение: {std_auc}")

print(f"\nЛучшее значение C: {best_C}")


C: 1e-06, Средний AUC: 0.698, Стандартное отклонение: 0.008
C: 0.001, Средний AUC: 0.996, Стандартное отклонение: 0.001
C: 1, Средний AUC: 1.0, Стандартное отклонение: 0.0

Лучшее значение C: 1


In [ ]:
#1. duration
#2. 0.99
#3. 0.465
#4. 0.02
#5. 0.0001
#6. 1